<a href="https://colab.research.google.com/github/Cattum/Infovis-Redesign/blob/main/redesign_visualization_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
pip install dash pandas plotly

In [13]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, dash_table, Input, Output, State, ctx

In [14]:
df_all = pd.read_csv('/content/climate-risk-index-1.csv')
df_all['id'] = df_all.index

app = Dash(__name__)

colorscale = [
    [0, "lightblue"],
    [0.5, "orange"],
    [1, "darkred"]
]

app.layout = html.Div([
    html.Div(html.Div([
    dcc.Graph(id='map', style={'height': '100vh', 'position': 'relative', 'zIndex': 0}),

    html.Div([
        html.Strong("Variable Description", style={'color': 'white'}),
        html.Ul([
            html.Li("cri_rank: Rank of the country on the Climate Risk Index", style={'color': 'lightgray'}),
            html.Li("cri_score: Score of the country on the Climate Risk Index", style={'color': 'lightgray'})
        ], style={'margin': '5px 0 0 0', 'padding': '0 0 0 20px'})
    ], style={
        'position': 'absolute',
        'top': '20px',
        'right': '30px',
        'backgroundColor': '#111111',
        'padding': '10px',
        'border': '1px solid #444',
        'borderRadius': '8px',
        'zIndex': 10,
        'maxWidth': '260px'
    })
], style={'position': 'relative'}), style={'width': '70%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '1%'}),


    html.Div([
        dcc.Input(
            id='search-input',
            type='text',
            placeholder='Search country...',
            style={
                'width': '100%',
                'marginBottom': '10px',
                'padding': '10px',
                'backgroundColor': '#222',
                'color': 'white',
                'border': '1px solid #555'
            }
        ),
        dash_table.DataTable(
            id='table',
            columns=[
                {"name": "country", "id": "country"},
                {"name": "cri_score", "id": "cri_score"},
                {"name": "cri_rank", "id": "cri_rank"}
            ],
            data=df_all.to_dict('records'),
            row_selectable='single',
            selected_rows=[],
            style_cell={
                'textAlign': 'left',
                'backgroundColor': '#111111',
                'color': 'white',
                'fontFamily': 'Arial',
                'fontSize': '14px',
                'padding': '10px'
            },
            style_header={
                'backgroundColor': '#222222',
                'fontWeight': 'bold',
                'color': 'lightgray'
            },
            style_table={
                'overflowY': 'scroll',
                'height': '92vh',
                'border': '1px solid #333',
            }
        )
    ], style={
        'width': '23%',
        'display': 'inline-block',
        'verticalAlign': 'top',
        'backgroundColor': '#111111',
        'padding': '1%',
        'color': 'white'
    })
], style={'backgroundColor': '#000000', 'height': '100vh'})

@app.callback(
    Output('map', 'figure'),
    Input('table', 'selected_rows'),
    State('table', 'data')
)
def update_map(selected_rows, table_data):
    dff = pd.DataFrame(table_data)
    fig = px.choropleth(
        dff,
        locations="country",
        locationmode="country names",
        color="cri_score",
        hover_name="country",
        hover_data=["cri_score", "cri_rank"],
        color_continuous_scale=colorscale,
        title="Climate Risk Index by Country",
    )

    if selected_rows:
        selected_country = dff.iloc[selected_rows[0]]["country"]
        fig.add_scattergeo(
            locations=[selected_country],
            locationmode="country names",
            text=[f"Selected: {selected_country}"],
            mode="markers+text",
            marker=dict(size=12, color="black"),
            textposition="top center",
            showlegend=False
        )

    fig.update_layout(
        geo=dict(showframe=False, showcoastlines=False),
        paper_bgcolor='black',
        plot_bgcolor='black',
        font=dict(color='white'),
        title_font=dict(color='white'),
        geo_bgcolor='black',
        margin=dict(l=0, r=0, t=40, b=0)
    )
    return fig

@app.callback(
    Output('table', 'selected_rows'),
    Input('map', 'clickData'),
    Input('table', 'selected_rows'),
    Input('table', 'active_cell'),
    State('table', 'data')
)
def sync_selection(map_click, selected_rows, active_cell, table_data):
    triggered = ctx.triggered_id
    if triggered == 'map' and map_click:
        clicked_country = map_click['points'][0]['location']
        for i, row in enumerate(table_data):
            if row['country'] == clicked_country:
                return [i]
    elif triggered == 'table':
        if selected_rows and active_cell and selected_rows[0] == active_cell['row']:
            return []
    return selected_rows

@app.callback(
    Output('table', 'data'),
    Input('search-input', 'value')
)
def filter_table(search_value):
    if not search_value:
        return df_all.to_dict('records')
    else:
        filtered_df = df_all[df_all['country'].str.contains(search_value, case=False, na=False)]
        return filtered_df.to_dict('records')

if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>